## MINIST的numpy实现

In [2]:
import numpy as np
from torchvision import datasets
from torchvision.transforms import ToTensor

In [52]:
# 利用pytorch下载的数据集，转换成numpy的ndarray
def get_data():
    training_data = datasets.MNIST(
        root="data",
        train=True,
        download=True,
        transform=ToTensor()
    )
    test_data = datasets.FashionMNIST(
        root="data",
        train=False,
        download=True,
        transform=ToTensor()
    )
    tr = training_data.data / 255
    te = training_data.targets
    return tr.numpy(), te.numpy()

tr, te = get_data()
features = tr.reshape(tr.shape[0], -1) # (60000, 784)

#labels符合交叉熵误差的形状
labels = np.zeros((features.shape[0], 10)) 
labels[np.arange(labels.shape[0]), te] = 1.0 

In [75]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# 
def softmax(x):
    if x.ndim == 1:
        x = x.reshape(1, -1)
    c = np.max(x, axis = 1, keepdims = True)
    exp_a = np.exp(x - c) # 减去c是为了防止溢出
    sum_exp_a = np.sum(exp_a, axis = 1, keepdims = True)
    y = exp_a / sum_exp_a
    return y
    
     
# 损失函数 
def cross_entropy_error(self, y, t):
    # 由于loge(0)是负无穷大-inf，计算机无法继续之后的运算
    # 所以给输入增加一个微小的数，并且不影响结果
    delta = 1e-7
    # 除以批量
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + delta)) / batch_size

# 定义网络
# 两层网络 第一层是affine层， W1: 748 * 100, B1: 100
# 激活函数Sigmoid
# 两层网络 第二层是affine层， W2: 100 * 10, B2: 10
# 输出函数是softmax 误差是交叉熵误差
class SimpleNet:
    
    def __init__(self, l1_size, l2_size, out_size):
        self.params = {
            'W1': np.random.rand(l1_size, l2_size), 
            'B1': np.zeros(l2_size),
            'W2': np.random.rand(l2_size, out_size), 
            'B2': np.zeros(out_size), 
        }

    def predict(self, x):
        w1, w2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['B1'], self.params['B2']
        
        a1 = x.dot(w1) + b1
        z1 = sigmoid(a1)
        
        a2 = z1.dot(w2) + b2
        y = softmax(a2)
        
        return y